# Cycle Cancelling Algorithm

## Library import
Let's start by importing the Library that will be used to initialize and draw the graph.
Before run the following cell you need to install the library:
* __Anaconda__ : conda install pydot
* __pip__ : pip install pydot
* __Arch__ : pacman -Syu python-pydot


In [1]:
import pydot

In [2]:
import numpy as np
import math
import copy
import matplotlib.pyplot as plt
from pylatex import Figure, SubFigure
from pylatex import Document, Package, NoEscape, Command, Chapter, Section, Subsection, TikZ
import os

In [3]:
infinite = 1000
tex_file = "ricerca"

doc = None

latex_point=None

graphs = []

## Graph creation

This function crates the graph, the graph can have extra attributes, for example the disponibility or the request.

The nodes must be positive integer or letter

In [4]:
def createGraph(nodes, edges, extraNodeAttributes=False) :
    graph = pydot.Dot(graph_type='digraph', layout='dot', nodesep='2', rankdir='LR')
    if(not extraNodeAttributes) :
        for node in nodes:
            graph.add_node(pydot.Node(node))
    else :
        for nodeId, attributes in nodes:
            node = pydot.Node(nodeId)
            for key, value in attributes.items():
                node.set(key, value)
            graph.add_node(node)
    for edge in edges:
        src, dst, weight, capacity, x = edge
        edge_obj = pydot.Edge(src, dst, label=f'{weight}/{capacity}/{x}')
        graph.add_edge(edge_obj)
        #edge_objects.append(edge_obj)

    return graph


## Add s and t nodes

This function get the next value, for example if value is A return B, but if number of graph's nodes are more than 25 returns a number

it is a way to avoid repetitions of nodes.

In [5]:
def next(valore, len):
    if valore.isdigit() or len > 25:
        if (len > 25) and not valore.isdigit() :
            print("Valore = 1")
            valore = 1
        res = str(int(valore) + 1)
        return res
    elif valore.isalpha():
        if valore.lower() == 'z':
            return 'A'
        else:
            return chr(ord(valore.lower()) + 1).upper()
    else:
        return None

Add extra nodes and edges

In [6]:
def addExtraNodes(graph) :
    nodes = graph.get_nodes()
    lastNode = nodes[len(nodes)-1].get_name()
    print(lastNode)
    sVal = next(lastNode, len(nodes) )
    while graph.get_node(sVal) != [] :
        sVal = next(sVal, len(nodes) )
    s = pydot.Node(sVal)
    print("TRovato s = ", s.get_name())
    while graph.get_node(sVal) != [] or s.get_name() == sVal :
        sVal = next(sVal, len(nodes)+1 )
    t = pydot.Node(sVal)
    print("Trovato t = ", t.get_name())
    s.set('b', 0)
    t.set('b', 0)
    for node in graph.get_nodes() :
        value = node.get('b')
        if value > 0 :
            edge = pydot.Edge(s.get_name(), node.get_name(), label=f'{0}/{value}/0')
            s.set('b', s.get('b') + value)
            graph.add_edge(edge)
        elif value < 0 :
            edge = pydot.Edge(node.get_name(),t.get_name(), label=f'{0}/{-value}/0')
            t.set('b', t.get('b') + value)
            graph.add_edge(edge)
    graph.add_node(s)
    graph.add_node(t)
    return graph

This function remove the extra nodes and relative edges

In [7]:
def removeExtraNodes(graph) :
    s = graph.get_nodes()[-2].get_name()
    t = graph.get_nodes()[-1].get_name()
    graph.del_node(s)
    graph.del_node(t)
    for node in graph.get_nodes() :
        del node.obj_dict['attributes']['exp']
        del node.obj_dict['attributes']['label']
    for edge in graph.get_edges() :
        if (edge.get_source() == s or edge.get_source() == t) or edge.get_destination() == s or edge.get_destination() == t :
            graph.del_edge(edge.get_source(), edge.get_destination())
    return graph

## Extra Functions

In [8]:
def printNodes(graph) :
    for node in graph.get_nodes() :
        print(node)

### Print Graph Nodes
This function print the graph's node

In [9]:
def printEdges(graph) :
    for edge in graph.get_edges() :
        src = edge.get_source()
        dst = edge.get_destination()
        label = edge.get_label()
        print(f"Arco da {src} a {dst} con etichetta: {label}")
    

### Correct direction
This function check the correct direction of the edge

In [10]:
def isCorrectDirection(graph, src, dst) :
    return len(graph.get_edge(src, dst)) == 1

## LATEX Functuons

In [11]:
def get_pdf() :
    global doc
    doc.generate_pdf(tex_file, clean_tex=False, compiler="lualatex")

In [146]:
def checkLatexFile() :
    global doc, latex_point
    if not os.path.exists(tex_file + '.tex') or doc == None :
        print("Entro")
        doc = Document(documentclass='report')
        doc.preamble.append(Package('tikz'))
        doc.preamble.append(Package('subcaption'))
        doc.preamble.append(Package('graphicx'))
        doc.preamble.append(NoEscape(r'\usetikzlibrary{graphs,graphdrawing, quotes}'))
        doc.preamble.append(NoEscape(r'\usetikzlibrary{graphs.standard, arrows.meta}'))
        doc.preamble.append(NoEscape(r'\usegdlibrary{force}'))
        doc.preamble.append(Package('geometry'))
        doc.preamble.append(NoEscape(r"""
\geometry{
    a4paper,
    left=2cm,
    right=2cm,
    top=2cm,
    bottom=2cm,    
}"""))
        doc.preamble.append(NoEscape(r'\title{Ricerca Operativa}'))
        doc.preamble.append(NoEscape(r'\author{luca carabini}'))

        doc.append( Command('maketitle') )
        doc.append( Command('tableofcontents') )
        latex_point = doc
        doc.generate_tex(tex_file)

### WRITE GRAPH

In [13]:
def findCuppleEdges(graph, src, dst) :
    if (len(graph.get_edge(src, dst)) > 0 ) and (len(graph.get_edge(dst, src)) > 0 ) :
        return ", bend left"
    else :
        return ""

In [14]:
def writeOnTex(text) :
    checkLatexFile()
    if latex_point != None :
        latex_point.append(NoEscape(text))
    else :
        doc.append(NoEscape(text))

In [147]:
def writeMultipleGraph() :
    global doc
    for caption, graph in graphs :
        with latex_point.create(Figure(position='htbp')) as figure :
            figure.append(NoEscape(r'\centering'))
            with figure.create(SubFigure()) as subfigure :
                with subfigure.create(TikZ(options=NoEscape(r'scale=0.8,every node/.style={transform shape},every edge quotes/.style={font=\small},>={Stealth[width=3mm,length=3mm]}'))) as tikz :
                    tikz.append(NoEscape(graph))
            if caption != "" :
                figure.append(NoEscape('\caption{' + caption + '}'))
    latex_point.append(NoEscape(r"\newpage"))

In [124]:
def getEdgeLabel(c, u, x, edge) :
    cEdge, uEdge, xEdge = edge.get_label().split('/')
    label = "("
    if c :
        label += cEdge
    if u:
        if len(label) > 1 :
            label += ","
        label += uEdge
    if x :
        if len(label) > 1 :
            label += ","
        label += xEdge
    label += ')'
    return label

In [111]:
def getNodeLabel(state, graph, node) :
    label = graph.get_node(node)[0].get('label')
    if state and label != None and label != () :
        node, val = label
        return r'[label=right:"' + node + "," + str(val) + '"]'
    else :
        return ""

In [119]:
def writeGraph(graph, label=False, path=[], c=False, u=False, x=False, caption="") :
    global graphs
    checkLatexFile()
    tikz = r"""
    \graph [
        spring layout,
        node distance=4cm,
        nodes={draw, circle, swap, fill=cyan!60},
        edge quotes={auto,sloped},
    ] {
    """
    for edge in graph.get_edges() :
        src = edge.get_source()
        dst = edge.get_destination()
        bend = findCuppleEdges(graph, src, dst)
        tikz += rf"""{src}{getNodeLabel(label, graph, src)} -> [ "{getEdgeLabel(c, u, x, edge)}" pos=0.3 {bend} ] {dst}{getNodeLabel(label, graph, dst)};
        """
    tikz += r"""
    }; """
    graphs.append( (caption,tikz) )

In [19]:
def startStopGraphWrite(start=True) :
    global graphs
    if start :
        graphs = []
    else :
        writeMultipleGraph()

In [20]:
def chapter(chapterName, text="") :
    global latex_point
    checkLatexFile()
    with doc.create(Chapter(chapterName)) as chapter:
        doc.append(text)
        latex_point = chapter

In [21]:
def section(sectionName, text="") :
    global latex_point
    checkLatexFile()
    with doc.create(Section(sectionName)) as section:
        doc.append(text)
        latex_point = section

In [22]:
def subsection(subName, text="") :
    global latex_point
    checkLatexFile()
    with doc.create(Subsection(subName)) as sub:
        sub.append(text)
        latex_point = sub

## Feasible flow

This function is the break clause for node labeling.

Return $\textbf{True}$ if all nodes that have a label are expanded

In [23]:
def stopClause(graph) :
    for node in graph.get_nodes() :
        if(len(node.get('label')) != 0 and node.get('exp') == False) :
            return False
    return True

This function initialize the label and the exp variable in each node

In [24]:
def initLabel(graph) :
    for node in graph.get_nodes() :
        node.set('exp', False)
        node.set('label', ())
    return graph

This function labels the nodes that are connected  to node i:
* if the edge have $\textbf{i}$ as $\textbf{source}$ the label will be $\textbf{i}$ and minimum between the delta of i and the capacity - x of the edge
* if the edge have $\textbf{i}$ as $\textbf{destination}$ the label will be $\textbf{-i}$ and minimum between the delta of i and the capacity - x of the edge

In [72]:
def getConnectedEdgs(graph, i) :
    src = i.get_name()
    prev, delta_i = i.get('label')
    for edge in graph.get_edges() :
        src_e = edge.get_source()
        dst = edge.get_destination()
        w,capacity, x = edge.get_label().split('/')
        if(src == src_e and graph.get_node(dst)[0].get('label') == ()) :
            if int(x) < int(capacity) :
                graph.get_node(dst)[0].set('label', (src, min(int(capacity)-int(x), int(delta_i))))
                writeGraph(graph = graph, u=True, x=True, caption=f"Etichetto {dst}", label=True)
                print(f"Etichetto {dst} con etichetta: {graph.get_node(dst)[0].get('label')}")
        elif dst == src and int(x) > 0 and graph.get_node(src_e)[0].get('label') == ():
            graph.get_node(src_e)[0].set('label', ('-' + src , min(int(capacity)-int(x), int(delta_i))))
            writeGraph(graph = graph, u=True, x=True, caption=f"Etichetto {src_e}", label=True)
            print(f"Etichetto {src_e} con etichetta: {graph.get_node(src_e)[0].get('label')}")
    graph.get_node(src)[0].set('exp', True)
    return graph

### Augmenting Path
This function obtains the augmenting Path 

In [45]:
def getAugmentingPath(graph) :
    s = graph.get_nodes()[-2]
    node = graph.get_nodes()[-1]
    src, d_t = node.get('label')
    while(node.get_name() != s.get_name()) :
        src, x = node.get('label')
        dst = node.get_name()
        if isCorrectDirection(graph, src, dst) :
            w,c,x_edge = graph.get_edge(src,dst)[0].get_label().split('/')
            x_edge_old = x_edge
            x_edge = int(x_edge) + int(d_t)
            graph.get_edge(src,dst)[0].set_label(f'{w}/{c}/{x_edge}')
        else :
            w,c,x_edge = graph.get_edge(dst,src)[0].get_label().split('/')
            x_edge = int(x_edge) - int(x)
            graph.get_edge(dst,src)[0].set_label(f'{w}/{c}/{x_edge}')
        node = graph.get_node(src)[0]
    return graph, d_t

This function labels the graph until all nodes that have a label are expanded

In [112]:
def augmentingPath(graph) :
    startStopGraphWrite()
    s = graph.get_nodes()[-2].get_name()
    graph.get_nodes()[-2].set('label', (s, infinite))
    writeGraph(graph = graph, u=True, x=True, caption=f"Etichetto {s}", label=True)
    while(not stopClause(graph)) :
        for node in graph.get_nodes() :
            label = node.get('label')
            #print(f"Node {node.get_name()} con label {label}")
            if(len(label) != 0 and node.get('exp') == False) :
                graph = getConnectedEdgs(graph, node)
    startStopGraphWrite(False)
    return graph

### Feasible Flow
This function find the feasible flow while until node t can be reached

In [44]:
def getFeasibleFlow(graph) :
    section('Ricerco il flusso ammissibile')
    addExtraNodes(graph)
    v = 0
    subsection('Iterazione 0')
    init_graph = initLabel(graph)
    new_graph = augmentingPath(init_graph)
    count=0
    while(graph.get_nodes()[-1].get_label() != ()) :
        new_graph, d_t = getAugmentingPath(graph)
        graph = new_graph
        v += d_t
        print("V = ", v)
        init_graph = initLabel(graph)
        count += 1
        subsection(f'Iterazione {count}')
        new_graph = augmentingPath(init_graph)
    removeExtraNodes(graph)
    printEdges(graph)
    return graph

## Find the Negative Path


### Bellman Ford
This a simple Bellman Ford algorithm, but I take the pred of each node to find the negative path 

In [29]:
def bellmanFord(graph, source):
    n = len(graph.get_nodes())
    d = {}
    p = {}
    for node in graph.get_nodes() :
        d[node.get_name()] = infinite
        p[node.get_name()] = -1
    d[source] = 0
    edges = graph.get_edges()
    edges.sort(key=lambda x: x.get_source())
    for k in range(n-1) :
        for edge in  edges:
            src = edge.get_source()
            dst = edge.get_destination()
            w,c,x = edge.get_label().split('/')
            if (d[dst] > d[src]+int(w)) :
                d[dst] = d[src]+int(w)
                p[dst] = src

    #Try to find a negative Path
    for edge in edges:
        src = edge.get_source()
        dst = edge.get_destination()
        w,c,x = edge.get_label().split('/')
        if (d[dst] > d[src]+int(w)) :
            #print("Cerco Ciclo neg")
            negativePath = [(src,dst)]
            temp = src
            count = 0
            print(p)
            while temp != dst and count < len(graph.get_edges()): 
                #print(f"{temp != dst},dst = {dst}, temp = {temp} ")
                count += 1
                #print("Aggiungo ", temp)
                negativePath.append((p[temp], temp))
                temp = p[temp]
                #print("New Temp ", temp)
            #Check if the negative path is correct
            if ( count < len(graph.get_edges())) :
                negativePath.reverse()
                #print(f"Trovato : {negativePath}")
                return negativePath
    return []


### Find Path Function
This function do the Bellman-Ford algorithm until it finds a Negative Path

In [30]:
def findNegPath(graph) :
    print("Inizio ricerca di ciclo negativo")
    for node in graph.get_nodes() :
        path = bellmanFord(graph, node.get_name())
        if len(path) > 0 :
            return path
    return []

## CYCLE

This function delete edges with 0 capacity

In [31]:
def delZeroCapacityEdges(graph) :
    for edge in graph.get_edges() :
        src = edge.get_source()
        dst = edge.get_destination()
        w,c,x = edge.get_label().split('/')
        if (int(c) <= 0) :
            print(f"rimuovo arco da {src} a {dst} con capacità {int(c)} e x {int(x)}")
            graph.del_edge(src, dst)
    return graph

This Function take a graph only with original edges, and return a new graph also with the back edges, based on $\textbf{x}$ parameter

In [32]:
def setBackEdge(graph) :
    for edge in graph.get_edges() :
        src = edge.get_source()
        dst = edge.get_destination()
        w,c,x = edge.get_label().split('/')
        if (int(x) > 0) :
            label = f"{-int(w)}/{int(x)}/{0}"
            if (len(graph.get_edge(dst, src)) > 0) :
                    graph.get_edge(dst, src)[0].set_label(label)
            else :
                backEdge = pydot.Edge(dst, src, label=label)
                graph.add_edge(backEdge)
            edge.set_label(f"{w}/{int(c)-int(x)}/{x}")
    return delZeroCapacityEdges(graph)            

This function increase or decrease the x of graph's edges, based of a delta, the negative path and its dircection

In [33]:
def updateX(graph, path, delta) :
    for src,dst in path :
        if isCorrectDirection(graph, src, dst) :
            w,c,x = graph.get_edge(src,dst)[0].get_label().split('/')
            x = int(x)
            x += delta
            graph.get_edge(src,dst)[0].set_label(f"{w}/{c}/{x}")
        else :
            w,c,x = graph.get_edge(dst,src)[0].get_label().split('/')
            x = int(x)
            x -= delta
            graph.get_edge(dst,src)[0].set_label(f"{w}/{c}/{x}")
    return graph

This function, return the correct delta, that is the minimum capacity of path

In [34]:
def getMinDelta(graph,path) :
    minDelta = infinite
    currentDelta = infinite
    print("Path",path)
    for src,dst in path :
        if isCorrectDirection(graph, src, dst) :
            w,c,x = graph.get_edge(src, dst)[0].get_label().split('/')
            currentDelta = int(c)
        else :
            w,c,x = graph.get_edge(dst, src)[0].get_label().split('/')
            currentDelta = int(c)
        if currentDelta < minDelta :
            minDelta = currentDelta
    return minDelta
            

This is the main function and performs the following operations:
* finds a $\textbf{feassible flow}$
* finds a negative path
* updates the x parameter of each node


In [41]:
def cycleCancelling(graph) :
    chapter('Cycle Cancelling')
    startStopGraphWrite()
    writeGraph(graph, c=True, u=True)
    startStopGraphWrite(False)
    xGraph = getFeasibleFlow(graph)
    stop = False
    count = 0
    while(not stop) :
        print("Iterazione ", count)
        count += 1
        new_graph = setBackEdge(copy.deepcopy(xGraph))
        path = findNegPath(new_graph)
        if (len(path) > 0) :
            delta = getMinDelta(new_graph, path)
            print("Delta = ", delta)
            updateX(xGraph, path, delta)
        else :
            stop = True
    print("Finito")
    printEdges(xGraph)
    get_pdf()
        

In [ ]:
nodes = [('1', {'b':4}),
              ('2', {'b':0}),
              ('3', {'b':0}),
              ('4', {'b':0}),
              ('5', {'b':0}),
              ('6', {'b':-4})
        ]
# Aggiungere archi con attributi di peso e capacità
edges = [ ('1', '2', 2, 3,0), ('1', '3', 5, 3, 0), 
          ('2', '3', 1, 1, 0), ('2', '4', 5, 1, 0), ('2', '5', 7, 4, 0), ('2', '1', 7, 4, 0), 
          ('3', '5', 2, 2, 0), 
          ('4', '6', 5, 3, 0),
          ('5', '4', 3, 2, 0), ('5', '6', 2, 3, 0) 
        ]
graph = createGraph(nodes, edges, True)
printNodes(graph)
cycleCancelling(graph)


1 [b=4];
2 [b=0];
3 [b=0];
4 [b=0];
5 [b=0];
6 [b=-4];
Entro
6
TRovato s =  7
Trovato t =  8
Etichetto 1 con etichetta: ('7', 4)
Etichetto 2 con etichetta: ('1', 3)
Etichetto 3 con etichetta: ('1', 3)
Etichetto 4 con etichetta: ('2', 1)
Etichetto 5 con etichetta: ('2', 3)
Etichetto 6 con etichetta: ('4', 1)
Etichetto 8 con etichetta: ('6', 1)
V =  1
Etichetto 1 con etichetta: ('7', 3)
Etichetto 2 con etichetta: ('1', 2)
Etichetto 3 con etichetta: ('1', 3)
Etichetto 5 con etichetta: ('2', 2)
Etichetto 4 con etichetta: ('5', 2)
Etichetto 6 con etichetta: ('5', 2)
Etichetto 8 con etichetta: ('6', 2)
V =  3
Etichetto 1 con etichetta: ('7', 1)
Etichetto 3 con etichetta: ('1', 1)
Etichetto 5 con etichetta: ('3', 1)
Etichetto 2 con etichetta: ('-5', 1)
Etichetto 4 con etichetta: ('5', 1)
Etichetto 6 con etichetta: ('5', 1)
Etichetto 8 con etichetta: ('6', 1)
V =  4
Arco da 1 a 2 con etichetta: 2/3/3
Arco da 1 a 3 con etichetta: 5/3/1
Arco da 2 a 3 con etichetta: 1/1/0
Arco da 2 a 4 con etiche

In [126]:
# Aggiungere nodi al grafo
nodes = [('1', {'b':4}),
              ('2', {'b':3}),
              ('3', {'b':0}),
              ('4', {'b':0}),
              ('5', {'b':-7})]
# Aggiungere archi con attributi di peso e capacità
edges = [ ('1', '3', 6, 4,0), ('1', '4', 5, 4, 0),
          ('2', '1', 1, 2, 0), ('2', '4', 4, 3, 0), ('2', '3', 3, 1, 0), 
          ('3', '4', 3, 2, 0), ('3', '5', 2, 3, 0), 
          ('4', '5', 5, 5, 0)
        ]
graph = createGraph(nodes, edges, True)
writeGraph(graph)
doc.generate_tex(tex_file)
cycleCancelling(graph)

Entro
5
TRovato s =  6
Trovato t =  7
Etichetto 1 con etichetta: ('6', 4)
Etichetto 2 con etichetta: ('6', 3)
Etichetto 3 con etichetta: ('1', 4)
Etichetto 4 con etichetta: ('1', 4)
Etichetto 5 con etichetta: ('3', 3)
Etichetto 7 con etichetta: ('5', 3)
V =  3
Etichetto 1 con etichetta: ('6', 1)
Etichetto 2 con etichetta: ('6', 3)
Etichetto 3 con etichetta: ('1', 1)
Etichetto 4 con etichetta: ('1', 1)
Etichetto 5 con etichetta: ('4', 1)
Etichetto 7 con etichetta: ('5', 1)
V =  4
Etichetto 2 con etichetta: ('6', 3)
Etichetto 1 con etichetta: ('2', 2)
Etichetto 4 con etichetta: ('2', 3)
Etichetto 3 con etichetta: ('2', 1)
Etichetto 5 con etichetta: ('4', 3)
Etichetto 7 con etichetta: ('5', 3)
V =  7
Arco da 1 a 3 con etichetta: 6/4/3
Arco da 1 a 4 con etichetta: 5/4/1
Arco da 2 a 1 con etichetta: 1/2/0
Arco da 2 a 4 con etichetta: 4/3/3
Arco da 2 a 3 con etichetta: 3/1/0
Arco da 3 a 4 con etichetta: 3/2/0
Arco da 3 a 5 con etichetta: 2/3/3
Arco da 4 a 5 con etichetta: 5/5/4
Iterazione  0

In [39]:
get_pdf()

In [40]:
# Creazione di un grafo diretto pesato con capacità sugli archi

# Aggiungere nodi al grafo
nodes = [('1', {'b':5}),
              ('2', {'b':-1}),
              ('3', {'b':-1}),
              ('4', {'b':-1}),
              ('5', {'b':-1}),
              ('6', {'b':-1})
        ]
# Aggiungere archi con attributi di peso e capacità
edges = [ ('1', '2', 2, 5,0), ('1', '3', 3, 5, 0), 
          ('2', '3', 2, 5, 0), ('2', '4', 4, 5, 0), ('2', '5', 3, 5, 0), 
          ('3', '5', 4, 5, 0), 
          ('4', '6', 3, 5, 0),
          ('5', '4', 1, 5, 0), ('5', '6', 2, 5, 0) 
        ]
graph = createGraph(nodes, edges, True)
#graph.get_edge('1','2')
writeGraph(graph, 1)
#cycleCanceling(graph)

In [41]:
from pylatex import Document, Package, NoEscape
# Percorso del file .tex esistente
checkLatexFile()
# Codice TikZ da aggiungere
tikz_code = r'''
\begin{tikzpicture}
\draw[gray, thick] (-1,2) -- (2,-4);
\draw[gray, thick] (-1,-1) -- (2,2);
\filldraw[black] (0,0) circle (2pt) node[anchor=west]{Intersection point};
\end{tikzpicture}

'''
file = 'ricerca.tex'
# Leggi il contenuto del file .tex esistente
with open(file, 'r') as f:
    tex_content = f.read()

# Trova la posizione dell'\end{document}
end_document_pos = tex_content.find(r'\end{document}')

# Se \end{document} è stato trovato, aggiungi il codice TikZ prima di esso
if end_document_pos != -1:
    new_tex_content = tex_content[:end_document_pos] + tikz_code + tex_content[end_document_pos:]
        # Sovrascrivi il file .tex esistente con il nuovo contenuto
    with open(file, 'w') as f:
        f.write(new_tex_content)
else:
    print("Attenzione: Impossibile trovare \\end{document} nel file .tex")

In [ ]:
graph.write_png('grafo.png')
graph.write_dot('grafo.dot')

# Percorso del file .dot
file_dot = "grafo.dot"

# Carica il file .dot
graphll = pydot.graph_from_dot_file(file_dot)
graphll = graphll[0]

# Itera attraverso gli archi nel grafo
for edge in graphll.get_edges():
    # Verifica se l'arco ha un attributo "pos"
    if 'pos' in edge.obj_dict['attributes']:
        pos_attribute = edge.obj_dict['attributes']['pos']
        # Verifica se l'attributo "pos" contiene più di due punti
        print(len(pos_attribute.split()))
        if isinstance(pos_attribute, str) and len(pos_attribute.split()) > 5:
            print("L'arco", edge.get_source(), "->", edge.get_destination(), "è curvo.")
            pos_attribute = edge.obj_dict['attributes']['pos']
            coordinates = pos_attribute.split(' ')[1:]  # Escludi il primo elemento (tipo di punto)
            # Calcola la pendenza delle rette tra il primo e l'ultimo punto intermedi
            x1, y1 = map(float, coordinates[0].split(','))
            x2, y2 = map(float, coordinates[-1].strip('"').split(','))
            m_curve = (y2 - y1) / (x2 - x1)
            # Calcola la pendenza delle tangenti alla curva nei punti finali dell'arco
            x0, y0 = map(float, coordinates[0].split(','))
            xn, yn = map(float, coordinates[-1].strip('"').split(','))
            m_tangent_start = (yn - y0) / (xn - x0)
            x0, y0 = map(float, coordinates[-3].split(','))
            xn, yn = map(float, coordinates[-2].strip('"').split(','))
            m_tangent_end = (yn - y0) / (xn - x0)
            # Calcola l'angolo di curvatura
            curvature_angle = angle_between_lines(m_tangent_start, m_tangent_end)
            print("L'angolo di curvatura dell'arco", edge.get_source(), "->", edge.get_destination(), "è:", curvature_angle)
        else:
            print("L'arco", edge.get_source(), "->", edge.get_destination(), "non è curvo.")
    else:
        print("L'arco", edge.get_source(), "->", edge.get_destination(), "non ha l'attributo 'pos'.")
# Accedi ai nodi e agli archi

for node in graphll.get_nodes():
    print("Nodo:", node.get_name())
    # Accedi alle posizioni dei nodi se presenti
    if node.get_attributes().get('pos'):
        print("Posizione:", node.get_attributes()['pos'])
    
for edge in graphll.get_edges():
    print("Arco:", edge.get_source(), "->", edge.get_destination())
    # Accedi alle posizioni degli archi se presenti
    if edge.get_attributes().get('pos'):
        print("Posizione:", edge.get_attributes()['pos'])

